In [82]:
import pandas as pd
import matplotlib as plt
import rasterio
import numpy as np

Crime data loading

In [83]:
df = pd.read_csv("C:/Users/User/PycharmProjects/SafeRoute/NYPD_Arrests_Data__Historic_.csv") # TODO change to relative path
df = df.replace('(null)', np.nan)
df = df.replace('nan', np.nan)
df = df.dropna(axis=0)
df

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
1,237354740,12/04/2021,153.0,RAPE 3,104.0,RAPE,PL 1302502,F,B,41,0.0,25-44,M,WHITE HISPANIC,1013232.0,236725.0,40.816392,-73.895296,POINT (-73.89529641399997 40.816391847000034)
2,236081433,11/09/2021,681.0,"CHILD, ENDANGERING WELFARE",233.0,SEX CRIMES,PL 2601001,M,Q,113,0.0,25-44,M,BLACK,1046367.0,186986.0,40.679700,-73.776047,POINT (-73.77604736799998 40.67970040800003)
4,192799737,01/26/2019,177.0,SEXUAL ABUSE,116.0,SEX CRIMES,PL 1306503,F,M,25,0.0,45-64,M,BLACK,1000555.0,230994.0,40.800694,-73.941109,POINT (-73.94110928599997 40.800694331000045)
7,236106641,11/10/2021,263.0,"ARSON 2,3,4",114.0,ARSON,PL 1501001,F,B,41,72.0,25-44,M,WHITE HISPANIC,1017934.0,232221.0,40.804013,-73.878332,POINT (-73.87833183299993 40.804012949000025)
8,238383628,12/28/2021,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",113.0,FORGERY,PL 1702500,F,Q,113,0.0,18-24,M,BLACK,1045482.0,191341.0,40.691660,-73.779199,POINT (-73.77919852099996 40.69166001700007)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5498645,255851599,12/13/2022,969.0,"TRAFFIC,UNCLASSIFIED INFRACTIO",881.0,OTHER TRAFFIC INFRACTION,VTL051101A,M,M,32,0.0,25-44,M,BLACK,1000188.0,238820.0,40.822167,-73.942415,POINT (-73.94241483 40.82216691)
5498646,256469262,12/16/2022,198.0,CRIMINAL CONTEMPT 1,126.0,MISCELLANEOUS PENAL LAW,PL 21551B3,F,B,46,0.0,25-44,M,BLACK,1011750.0,250274.0,40.853578,-73.900591,POINT (-73.900591 40.853578)
5498647,260305039,12/20/2022,548.0,"GAMBLING, DEVICE, POSSESSION",350.0,GAMBLING,PL 2253001,M,K,72,0.0,25-44,F,ASIAN / PACIFIC ISLANDER,981944.0,171869.0,40.638420,-74.008307,POINT (-74.008307 40.63842)
5498648,255843041,12/13/2022,101.0,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,M,5,0.0,45-64,M,ASIAN / PACIFIC ISLANDER,986159.0,200104.0,40.715917,-73.993111,POINT (-73.993111 40.715917)


ALL OFFENCE DESCRIPTION

In [84]:
# np.unique(np.array())
list(set(df['OFNS_DESC']))

["BURGLAR'S TOOLS",
 'KIDNAPPING & RELATED OFFENSES',
 'ESCAPE 3',
 'FRAUDULENT ACCOSTING',
 'DANGEROUS WEAPONS',
 'NYS LAWS-UNCLASSIFIED VIOLATION',
 'LOITERING',
 'OFFENSES AGAINST PUBLIC ADMINI',
 'MURDER & NON-NEGL. MANSLAUGHTER',
 'F.C.A. P.I.N.O.S.',
 'HOMICIDE-NEGLIGENT,UNCLASSIFIE',
 'CRIMINAL MISCHIEF & RELATED OF',
 'ASSAULT 3 & RELATED OFFENSES',
 'PETIT LARCENY',
 'THEFT OF SERVICES',
 'OTHER OFFENSES RELATED TO THEFT',
 'POSSESSION OF STOLEN PROPERTY',
 'UNLAWFUL POSS. WEAP. ON SCHOOL',
 'OFFENSES RELATED TO CHILDREN',
 'OTHER TRAFFIC INFRACTION',
 'LOITERING,BEGGING',
 'FORCIBLE TOUCHING',
 'HARASSMENT',
 'OFF. AGNST PUB ORD SENSBLTY &',
 'MISCELLANEOUS PENAL LAW',
 'ALCOHOLIC BEVERAGE CONTROL LAW',
 'ANTICIPATORY OFFENSES',
 'ARSON',
 'ABORTION',
 'AGRICULTURE & MRKTS LAW-UNCLASSIFIED',
 'GRAND LARCENY',
 'LOITERING FOR DRUG PURPOSES',
 'FORGERY',
 'UNLAWFUL POSS. WEAP. ON SCHOOL GROUNDS',
 'OFFENSES AGAINST MARRIAGE UNCLASSIFIED',
 'HOMICIDE-NEGLIGENT,UNCLASSIFIED',
 'G

ASSIGNING ARBITRARY DANGER VALUES FOR CRIME

In [85]:
danger_scores = {
    'HOMICIDE-NEGLIGENT,UNCLASSIFIED': 10,
    'SEX CRIMES': 9,
    'GRAND LARCENY': 3,
    'ROBBERY': 8,
    'UNLAWFUL POSS. WEAP. ON SCHOOL GROUNDS': 6,
    'KIDNAPPING & RELATED OFFENSES': 9,
    'BURGLARY': 6,
    'DANGEROUS DRUGS': 5,
    'FRAUDS': 2,
    'RAPE': 12,
    'FELONY SEX CRIMES': 10,
    'ASSAULT 3 & RELATED OFFENSES': 8,
    'UNAUTHORIZED USE OF A VEHICLE 3 (UUV)': 4,
    'FELONY ASSAULT': 9,
    'MURDER & NON-NEGL. MANSLAUGHTER': 15,
    'THEFT-FRAUD': 3,
    'PETIT LARCENY': 2
}

FORMING NEW DATAFRAME

In [86]:
new_df = df.loc[df['OFNS_DESC'].isin(danger_scores.keys()), ['ARREST_DATE', 'Latitude', 'Longitude']]
new_df

,ARREST_DATE,Latitude,Longitude
1,12/04/2021,40.816392,-73.895296
2,11/09/2021,40.679700,-73.776047
4,01/26/2019,40.800694,-73.941109
9,01/06/2016,40.648650,-73.950336
10,12/04/2021,40.688584,-73.916526
...,...,...,...
5498636,12/11/2022,40.637301,-74.009474
5498639,12/30/2022,40.671104,-73.913562
5498641,12/13/2022,40.742954,-73.955274
5498648,12/13/2022,40.715917,-73.993111


In [87]:
new_df['Danger Score'] = df.loc[df['OFNS_DESC'].isin(danger_scores.keys()), 'OFNS_DESC'].map(danger_scores)
new_df

,ARREST_DATE,Latitude,Longitude,Danger Score
1,12/04/2021,40.816392,-73.895296,12
2,11/09/2021,40.679700,-73.776047,9
4,01/26/2019,40.800694,-73.941109,9
9,01/06/2016,40.648650,-73.950336,12
10,12/04/2021,40.688584,-73.916526,8
...,...,...,...,...
5498636,12/11/2022,40.637301,-74.009474,5
5498639,12/30/2022,40.671104,-73.913562,8
5498641,12/13/2022,40.742954,-73.955274,9
5498648,12/13/2022,40.715917,-73.993111,8


In [88]:
new_df.sort_values(by=['Longitude'])

,ARREST_DATE,Latitude,Longitude,Danger Score
3031914,03/27/2012,40.507709,-74.254939,5
3015760,03/27/2012,40.507709,-74.254939,5
2948228,06/05/2012,40.508303,-74.254560,8
3287428,08/02/2011,40.508303,-74.254560,9
3291149,07/20/2011,40.508303,-74.254560,5
...,...,...,...,...
3836331,04/23/2010,51.627063,-73.681780,8
3283455,07/30/2011,51.627063,-73.681780,8
3965562,12/11/2009,51.627063,-73.681780,5
4226739,05/22/2009,51.627063,-73.681780,5


In [89]:
(new_df['Latitude'].min(), new_df['Latitude'].max(), new_df['Longitude'].min(), new_df['Longitude'].max())

(40.49890536300006, 62.08307497800007, -74.25493873599999, -73.68178026799995)

OPENING CLUSTERED GEOTIF POPULATION FILE

In [90]:
with rasterio.open("C:/Users/User/PycharmProjects/SafeRoute/clustered.tif") as src:
    data = src.read()
    image = src.read(1)
    clusters = src.read(2)
    bounds = src.bounds
    

In [91]:
bounds

BoundingBox(left=-74.27999999999999, bottom=40.400000004, right=-73.66000000247999, top=41.4)

In [92]:
(abs(bounds[2] - bounds[0]), abs(bounds[3] - bounds[1]))

(0.6199999975199972, 0.9999999959999997)

In [93]:
step = (abs(bounds[2] - bounds[0])/np.shape(clusters)[1] + abs(bounds[3] - bounds[1])/np.shape(clusters)[0]) / 2
step

0.000833333329999998

COUNTING POPULATION PER CLUSTER

In [94]:
pop_dict = {c: [] for c in np.unique(clusters)}
pop_count = {c: 0 for c in np.unique(clusters)}
for iy, ix in np.ndindex(image.shape):
    ap = image[iy, ix] if image[iy, ix] > 0 else 0
    pop_dict[clusters[iy, ix]].append(ap)
for k, v in pop_dict.items():
    pop_count[k] = sum(pop_dict[k])/len(pop_dict[k]) if len(pop_dict[k]) > 0 else 0
pop_count

{-1: 18.903218407697636,
 0: 0.0,
 1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.0,
 18: 0.0,
 19: 0.0,
 20: 0.0,
 21: 0.0,
 22: 0.0,
 23: 0.0,
 24: 0.0,
 25: 0.0,
 26: 0.0,
 27: 0.0,
 28: 0.0,
 29: 0.0,
 30: 0.0,
 31: 0.0,
 32: 0.0,
 33: 0.0,
 34: 0.0,
 35: 0.0,
 36: 0.0,
 37: 0.0,
 38: 0.0,
 39: 548.4461538461538,
 40: 530.2962962962963,
 41: 0.0,
 42: 0.0,
 43: 0.0,
 44: 500.6,
 45: 474.61764705882354,
 46: 0.0,
 47: 453.8755980861244,
 48: 413.972602739726,
 49: 389.5257731958763,
 50: 375.33718244803697,
 51: 351.48130841121497,
 52: 336.6818181818182,
 53: 322.03021148036254,
 54: 329.2911392405063,
 55: 0.0,
 56: 0.0,
 57: 0.0,
 58: 292.8,
 59: 305.22222222222223,
 60: 314.38961038961037,
 61: 309.81739130434784,
 62: 296.79487179487177,
 63: 301.2952380952381,
 64: 289.0615384615385,
 65: 281.21875,
 66: 0.0,
 67: 0.0,
 68: 256.1044061302682,
 69: 267.7704918032787,
 7

In [95]:
cluster_score = {c: 0 for c in np.unique(clusters)}
cluster_crime_num = {c: 0 for c in np.unique(clusters)}
cluster_score

{-1: 0,
 0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 20: 0,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 0,
 27: 0,
 28: 0,
 29: 0,
 30: 0,
 31: 0,
 32: 0,
 33: 0,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 0,
 40: 0,
 41: 0,
 42: 0,
 43: 0,
 44: 0,
 45: 0,
 46: 0,
 47: 0,
 48: 0,
 49: 0,
 50: 0,
 51: 0,
 52: 0,
 53: 0,
 54: 0,
 55: 0,
 56: 0,
 57: 0,
 58: 0,
 59: 0,
 60: 0,
 61: 0,
 62: 0,
 63: 0,
 64: 0,
 65: 0,
 66: 0,
 67: 0,
 68: 0,
 69: 0,
 70: 0,
 71: 0,
 72: 0,
 73: 0,
 74: 0,
 75: 0,
 76: 0,
 77: 0,
 78: 0,
 79: 0,
 80: 0,
 81: 0,
 82: 0,
 83: 0,
 84: 0,
 85: 0,
 86: 0,
 87: 0,
 88: 0,
 89: 0,
 90: 0,
 91: 0,
 92: 0,
 93: 0,
 94: 0,
 95: 0,
 96: 0,
 97: 0,
 98: 0,
 99: 0,
 100: 0,
 101: 0,
 102: 0,
 103: 0,
 104: 0,
 105: 0,
 106: 0,
 107: 0,
 108: 0,
 109: 0,
 110: 0,
 111: 0,
 112: 0,
 113: 0,
 114: 0,
 115: 0,
 116: 0,
 117: 0,
 118: 0,
 119: 0,
 120: 0,
 121: 0,
 122

CALCULATING CRIME NUMBER AND SCORE PER CLUSTER

In [96]:
for row in new_df.itertuples():
    p = (row.Longitude, row.Latitude) # point declaration
    if abs(p[0])>abs(bounds[0]) or abs(p[0])<abs(bounds[2]): 
        continue
    elif abs(p[1])<abs(bounds[1]) or abs(p[1])>abs(bounds[3]): 
        continue
    cluster_score[clusters[round((abs(p[1]) - abs(bounds[1]))/step) - 1, round((abs(p[0]) - abs(bounds[2]))/step) - 1]] += row._4
    cluster_crime_num[clusters[round((abs(p[1]) - abs(bounds[1]))/step) - 1, round((abs(p[0]) - abs(bounds[2]))/step) - 1]] += 1

In [97]:
cluster_score

{-1: 11855977,
 0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 3711,
 5: 0,
 6: 0,
 7: 135,
 8: 16,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 7584,
 17: 291580,
 18: 0,
 19: 2677,
 20: 37831,
 21: 236,
 22: 0,
 23: 3990,
 24: 12678,
 25: 153,
 26: 0,
 27: 1276,
 28: 0,
 29: 59418,
 30: 0,
 31: 0,
 32: 0,
 33: 0,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 0,
 40: 0,
 41: 0,
 42: 0,
 43: 0,
 44: 0,
 45: 0,
 46: 0,
 47: 0,
 48: 0,
 49: 0,
 50: 0,
 51: 0,
 52: 0,
 53: 0,
 54: 0,
 55: 0,
 56: 0,
 57: 0,
 58: 0,
 59: 0,
 60: 0,
 61: 0,
 62: 0,
 63: 0,
 64: 0,
 65: 0,
 66: 307765,
 67: 0,
 68: 0,
 69: 0,
 70: 0,
 71: 0,
 72: 0,
 73: 0,
 74: 0,
 75: 0,
 76: 0,
 77: 0,
 78: 0,
 79: 0,
 80: 0,
 81: 0,
 82: 0,
 83: 0,
 84: 0,
 85: 0,
 86: 0,
 87: 0,
 88: 0,
 89: 0,
 90: 0,
 91: 0,
 92: 0,
 93: 0,
 94: 0,
 95: 0,
 96: 0,
 97: 0,
 98: 0,
 99: 0,
 100: 0,
 101: 0,
 102: 0,
 103: 0,
 104: 0,
 105: 0,
 106: 0,
 107: 0,
 108: 0,
 109: 0,
 110: 0,
 111: 0,
 112: 0,
 113: 0,
 114: 0,
 115: 0,
 116: 0

In [98]:
cluster_score[-1] = 0
cluster_crime_num[-1] = 0
cluster_score

{-1: 0,
 0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 3711,
 5: 0,
 6: 0,
 7: 135,
 8: 16,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 7584,
 17: 291580,
 18: 0,
 19: 2677,
 20: 37831,
 21: 236,
 22: 0,
 23: 3990,
 24: 12678,
 25: 153,
 26: 0,
 27: 1276,
 28: 0,
 29: 59418,
 30: 0,
 31: 0,
 32: 0,
 33: 0,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 0,
 40: 0,
 41: 0,
 42: 0,
 43: 0,
 44: 0,
 45: 0,
 46: 0,
 47: 0,
 48: 0,
 49: 0,
 50: 0,
 51: 0,
 52: 0,
 53: 0,
 54: 0,
 55: 0,
 56: 0,
 57: 0,
 58: 0,
 59: 0,
 60: 0,
 61: 0,
 62: 0,
 63: 0,
 64: 0,
 65: 0,
 66: 307765,
 67: 0,
 68: 0,
 69: 0,
 70: 0,
 71: 0,
 72: 0,
 73: 0,
 74: 0,
 75: 0,
 76: 0,
 77: 0,
 78: 0,
 79: 0,
 80: 0,
 81: 0,
 82: 0,
 83: 0,
 84: 0,
 85: 0,
 86: 0,
 87: 0,
 88: 0,
 89: 0,
 90: 0,
 91: 0,
 92: 0,
 93: 0,
 94: 0,
 95: 0,
 96: 0,
 97: 0,
 98: 0,
 99: 0,
 100: 0,
 101: 0,
 102: 0,
 103: 0,
 104: 0,
 105: 0,
 106: 0,
 107: 0,
 108: 0,
 109: 0,
 110: 0,
 111: 0,
 112: 0,
 113: 0,
 114: 0,
 115: 0,
 116: 0,
 117:

FINDING AREA PER CLUSTER

In [99]:
cluster_area = {c: (np.count_nonzero(clusters == c))*100 for c in np.unique(clusters)}
cluster_area

{-1: 44907300,
 0: 15700,
 1: 7300,
 2: 104500,
 3: 5600,
 4: 10000,
 5: 36100,
 6: 18200,
 7: 5200,
 8: 15500,
 9: 7100,
 10: 15700,
 11: 31900,
 12: 27300,
 13: 7600,
 14: 20300,
 15: 27200,
 16: 46100,
 17: 24700,
 18: 14200,
 19: 6900,
 20: 38700,
 21: 9100,
 22: 6800,
 23: 31600,
 24: 14200,
 25: 8500,
 26: 13800,
 27: 13400,
 28: 21300,
 29: 48900,
 30: 5500,
 31: 13284000,
 32: 55100,
 33: 12400,
 34: 6500,
 35: 7300,
 36: 5100,
 37: 2205900,
 38: 18700,
 39: 6500,
 40: 5400,
 41: 16900,
 42: 6500,
 43: 210600,
 44: 20000,
 45: 17000,
 46: 60400,
 47: 20900,
 48: 80300,
 49: 9700,
 50: 43300,
 51: 64200,
 52: 15400,
 53: 33100,
 54: 7900,
 55: 18600,
 56: 58900,
 57: 5900,
 58: 9500,
 59: 6300,
 60: 7700,
 61: 11500,
 62: 7800,
 63: 10500,
 64: 13000,
 65: 48000,
 66: 2539800,
 67: 11800,
 68: 104400,
 69: 6100,
 70: 12100,
 71: 39100,
 72: 170500,
 73: 19200,
 74: 14000,
 75: 10000,
 76: 12400,
 77: 161100,
 78: 8400,
 79: 146700,
 80: 19900,
 81: 49300,
 82: 11700,
 83: 49200,

In [100]:
for k, v in cluster_score.items():
    cluster_score[k] = v/cluster_area[k]
    cluster_crime_num[k] = cluster_crime_num[k]/cluster_area[k]

cluster_score

{-1: 0.0,
 0: 0.0,
 1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.3711,
 5: 0.0,
 6: 0.0,
 7: 0.025961538461538463,
 8: 0.001032258064516129,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.1645119305856833,
 17: 11.804858299595141,
 18: 0.0,
 19: 0.3879710144927536,
 20: 0.9775452196382429,
 21: 0.025934065934065935,
 22: 0.0,
 23: 0.12626582278481013,
 24: 0.8928169014084507,
 25: 0.018,
 26: 0.0,
 27: 0.09522388059701492,
 28: 0.0,
 29: 1.2150920245398773,
 30: 0.0,
 31: 0.0,
 32: 0.0,
 33: 0.0,
 34: 0.0,
 35: 0.0,
 36: 0.0,
 37: 0.0,
 38: 0.0,
 39: 0.0,
 40: 0.0,
 41: 0.0,
 42: 0.0,
 43: 0.0,
 44: 0.0,
 45: 0.0,
 46: 0.0,
 47: 0.0,
 48: 0.0,
 49: 0.0,
 50: 0.0,
 51: 0.0,
 52: 0.0,
 53: 0.0,
 54: 0.0,
 55: 0.0,
 56: 0.0,
 57: 0.0,
 58: 0.0,
 59: 0.0,
 60: 0.0,
 61: 0.0,
 62: 0.0,
 63: 0.0,
 64: 0.0,
 65: 0.0,
 66: 0.1211768643200252,
 67: 0.0,
 68: 0.0,
 69: 0.0,
 70: 0.0,
 71: 0.0,
 72: 0.0,
 73: 0.0,
 74: 0.0,
 75: 0.0,
 76: 0.0,
 77: 0.0,
 78: 0.0,
 79: 0.0,
 80: 0.

In [101]:
fin_df = pd.DataFrame({"pop_count_adj": pop_count ,'crime_number_adj': cluster_crime_num, 'crime_score_adj': cluster_score})
fin_df.to_csv('crime_score.csv', index_label='cluster')
fin_df

,pop_count_adj,crime_number_adj,crime_score_adj
-1,18.903218,0.000000,0.000000
0,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000
...,...,...,...
773,0.000000,0.153168,0.908911
774,0.000000,0.000000,0.000000
775,0.000000,0.000000,0.000000
776,0.000000,0.003133,0.019643


MERGING DATAFRAMES FOR FINAL FILE

In [102]:
df1 = pd.read_csv('crime_score.csv')
df2 = pd.read_csv('average_income.csv')
merged_df = pd.merge(df1, df2, on='cluster', how='outer')
merged_df.to_csv('all_data.csv', index=False)


ADJUSTING INCOME

In [103]:
edited_df = merged_df[(merged_df != 0).all(axis=1)]
i_l = edited_df['income'].tolist()
pop_list = edited_df['pop_count_adj'].tolist()
crime_list = edited_df['crime_score_adj'].tolist()

In [104]:
v_min = min([value for value in i_l if value >= 1000])
v_max = max([value for value in i_l if value >= 1000])
for i in range(len(i_l)):
    i_l[i] = ((i_l[i] - v_min) if v_min < i_l[i] else 0)
    i_l[i] = i_l[i]/v_max

In [105]:
v_max = max(pop_list)
for i in range(len(pop_list)):
    pop_list[i] = pop_list[i]/v_max

In [106]:
v_max = max(crime_list)
v_min = min(crime_list)
for i in range(len(crime_list)):
    crime_list[i] = (crime_list[i] - v_min)/v_max

In [111]:
edited_df['income'] = i_l
edited_df['pop_count_adj'] = pop_list
edited_df['crime_score_adj'] = pop_list
edited_df.to_csv('edited_data.csv', index=False)

C:\Users\User\AppData\Local\Temp\ipykernel_23964\3446773729.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edited_df['income'] = i_l
C:\Users\User\AppData\Local\Temp\ipykernel_23964\3446773729.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edited_df['pop_count_adj'] = pop_list
C:\Users\User\AppData\Local\Temp\ipykernel_23964\3446773729.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [110]:
df666 = pd.read_csv('edited_data.csv')
df666

,crime_number_adj,crime_score_adj,income
0,0.066221,1.000000,0.123698
1,0.067606,0.938490,0.146093
2,0.113624,0.997628,0.112600
3,0.097554,0.928987,0.184596
4,0.109884,0.911602,0.166143
...,...,...,...
2725,0.047255,0.072785,0.735745
2726,0.030858,0.095200,0.794781
2727,0.108132,0.113158,0.779728
2728,0.047692,0.101667,0.748260
